# Matrix Product States (MPS)

In [1]:
# Import necessary packages
import numpy as np
import tensornetwork as tn

In [2]:
# Create a random tensor
def block(*dim):
    '''Construct a tensor with random numbers from 0 to 1'''
    size = tuple([x for x in dim])
    return np.random.random_sample(size)


# Create MPS
def create_MPS(r, d, D):
    """ Create a MPS using the 'block' function to initialize tensors.

    Args:
        r (int): the rank of the TN (number of dangling indeces, and therefore of elementary tensors)
        d (int): physical dimension (dimension of the dangling indeces)
        D (int): bond dimension (dimension of the connected indeces)

    Return:
        mps: list of nodes
        conn_edges: list of connected edges
        
    """
    # create mps as a list of nodes
    mps = (
        [tn.Node(block(d, D), name=f'state_{0}')] + 
        [tn.Node(block(D, d, D), name=f'state_{i}') for i in range(1,r-1)] + 
        [tn.Node(block(D, d), name=f'state_{r-1}')]
    )

    # connect edges to form MPS
    conn_edges = []
    conn = mps[0][1] ^ mps[1][0]
    conn_edges.append(conn)
    for k in range(1,r-1):
        conn = mps[k][2] ^ mps[k+1][0]
        conn_edges.append(conn)

    # return
    return mps, conn_edges


# Create Matrix Product Operator (MPO)
def create_MPO(r, d, D):
    """ Create a MPO using the 'block' function to initialize tensors.

    Args:
        r (int): the rank of the TN (twice the number of dangling indeces, and therefore twice the number of elementary tensors)
        d (int): physical dimension (dimension of the dangling indeces)
        D (int): bond dimension (dimension of the connected indeces)

    Return:
        mpo: list of nodes
        conn_edges: list of connected edges
        
    """
    # create mpo as a list of nodes
    mpo = (
        [tn.Node(block(d, d, D), name=f'oper_{0}')] + 
        [tn.Node(block(D, d, D, d), name=f'oper_{i}') for i in range(1,r-1)] + 
        [tn.Node(block(D, d, d), name=f'oper_{r-1}')]
    )

    # connect edges to form MPO
    conn_edges = []
    conn = mpo[0][2] ^ mpo[1][0]
    conn_edges.append(conn)
    for k in range(1,r-1):
        conn = mpo[k][2] ^ mpo[k+1][0]
        conn_edges.append(conn)
    
    # return
    return mpo, conn_edges


# Contract a MPS into a single tensor
def contract_MPS(mps, edges):
    # sequential contractions
    for k in range(len(edges)):
        A = tn.contract(edges[k])
    return A

In [3]:
r, d, D = 16, 2, 3
mps, edges = create_MPS(r, d, D)

res = contract_MPS(mps, edges)
res.set_name('res')
res.get_rank()




16

In [4]:
# # sequential contractions
# for k in range(len(edges)):
#         res = tn.contract(edges[k])

# res = tn.contractors.auto([mps[i] for i in range(r)], ignore_edge_order=True)
# res = contract_MPS(mps)
# res.set_name('contraction')

a = [0,1,2,3,4,5,6,7,8,9]
a[::3]


[0, 3, 6, 9]